# Ex 1: Wedge

In [ ]:
from utils.FEniCSx_solver import FEniCSx_solver
from dolfinx import mesh as msh, fem, default_scalar_type, la
from mpi4py import MPI
import ufl
import numpy as np

import scipy

nx=8
ny=8
comm = MPI.COMM_WORLD
cell_type=msh.CellType.quadrilateral
mesh = msh.create_unit_square(comm=comm, nx=nx, ny=ny, cell_type=cell_type)

Wh = fem.functionspace(mesh, ('P', 1))
uh = fem.Function(Wh)
mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
boundary_facets = msh.exterior_facet_indices(mesh.topology)
boundary_dofs = fem.locate_dofs_topological(Wh, mesh.topology.dim-1, boundary_facets)
eps = fem.Constant(mesh, default_scalar_type(1e-8))
b = ufl.as_vector((fem.Constant(mesh, default_scalar_type(1.0)),fem.Constant(mesh, default_scalar_type(0.0))))
c = fem.Constant(mesh, default_scalar_type(0.0))
f = fem.Constant(mesh, default_scalar_type(1.0))
bcs = [fem.dirichletbc(fem.Constant(mesh, default_scalar_type(0.0)), boundary_dofs, Wh)]


pde_data = mesh,Wh,uh,eps,b,None,f,None,bcs



cid_lims = mesh.topology.index_map(2).local_range
marker_ids = np.arange(cid_lims[0], cid_lims[1])

for index in range(cid_lims[0],cid_lims[1]):
    if np.intersect1d(Wh.dofmap.cell_dofs(index), bcs[0].dof_indices()[0]).size > 0:
            marker_ids = marker_ids[marker_ids!=index]

marker = np.ones_like(marker_ids, dtype=np.int32)
cell_tag = msh.meshtags(mesh, mesh.topology.dim, marker_ids, marker)
dx = ufl.Measure("dx", domain=mesh, subdomain_data=cell_tag, subdomain_id=1)

x = ufl.SpatialCoordinate(mesh)
ex_exp = x[0]*(1-ufl.exp(-(1-x[0])/eps))* (1 - ((ufl.exp(-(1-x[1])/eps)  + ufl.exp(-(x[1])/eps))- ufl.exp(-1/eps))/(1-ufl.exp(-1/eps)))

exp = fem.Expression(ex_exp, Wh.element.interpolation_points())

u_ex = fem.Function(Wh)
u_ex.interpolate(exp)

n=ufl.FacetNormal(mesh)
residual = (-eps*ufl.div(ufl.grad(uh)) + ufl.dot(b, ufl.grad(uh)) + c * uh - f)**2 * dx

b_perp = ufl.as_vector((fem.Constant(mesh, default_scalar_type(0.0)),fem.Constant(mesh, default_scalar_type(-1.0))))
cross = abs(ufl.dot(b_perp, ufl.grad(uh)))
crosswind_loss = ufl.conditional(ufl.lt(cross, 1), 1/2*(5*cross**2 - 3*cross**3), ufl.sqrt(cross)) * dx
#loss = (uh-u_ex)**2 * ufl.dx
loss = residual + crosswind_loss
fs = FEniCSx_solver(pde_data=pde_data, loss_form=loss)


norm_b = ufl.sqrt(ufl.dot(b,b))
h = ufl.CellDiameter(domain=mesh) 
alpha = norm_b*h/(2*eps)
Xi = (1/ufl.tanh(alpha)-1/alpha)
tau_K = h/(2*norm_b)*Xi
Th = fem.functionspace(mesh, ('DG', 0))
tau = fem.Function(Th)
tau_exp = fem.Expression(tau_K, Th.element.interpolation_points())
tau.interpolate(tau_exp)
fs.set_weights(tau.x.array)


arr = np.array([])

upper = 100*fs.yh.x.array

# Ex2: Cylinder

In [5]:
from utils.FEniCSx_solver import FEniCSx_solver
from dolfinx import mesh as msh, fem, default_scalar_type, la
from mpi4py import MPI
import ufl
import numpy as np

import scipy

nx=8
ny=8
comm = MPI.COMM_WORLD
cell_type=msh.CellType.quadrilateral
mesh = msh.create_unit_square(comm=comm, nx=nx, ny=ny, cell_type=cell_type)

x = ufl.SpatialCoordinate(mesh)
i = ufl.Index()
Wh = fem.functionspace(mesh, ('P', 1))
uh = fem.Function(Wh)
mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
boundary_facets = msh.exterior_facet_indices(mesh.topology)
boundary_dofs = fem.locate_dofs_topological(Wh, mesh.topology.dim-1, boundary_facets)
eps = fem.Constant(mesh, default_scalar_type(10**(-3)))
b = ufl.as_vector((fem.Constant(mesh, default_scalar_type(2.0)),fem.Constant(mesh, default_scalar_type(3.0))))
c = fem.Constant(mesh, default_scalar_type(0.0))
expr = 16*x[0]*(1-x[0])*x[1]*(1-x[1])*(1/2+ufl.atan(2*eps**(-1/2)*(0.25**2-(x[0]-0.5)**2-(x[1]-1/2)**2))/ufl.pi)
f = -eps * expr.dx(i).dx(i) + b[i]*expr.dx(i) + c*expr
u_exact = fem.Expression(expr, Wh.element.interpolation_points())
uD = fem.Function(Wh)

uD.interpolate(u_exact)

bcs = [fem.dirichletbc(uD, boundary_dofs)]

pde_data = mesh,Wh,uh,eps,b,None,f,None,bcs



cid_lims = mesh.topology.index_map(2).local_range
marker_ids = np.arange(cid_lims[0], cid_lims[1])

for index in range(cid_lims[0],cid_lims[1]):
    if np.intersect1d(Wh.dofmap.cell_dofs(index), bcs[0].dof_indices()[0]).size > 0:
            marker_ids = marker_ids[marker_ids!=index]

marker = np.ones_like(marker_ids, dtype=np.int32)
cell_tag = msh.meshtags(mesh, mesh.topology.dim, marker_ids, marker)
dx = ufl.Measure("dx", domain=mesh, subdomain_data=cell_tag, subdomain_id=1)

residual = (-eps*ufl.div(ufl.grad(uh)) + ufl.dot(b, ufl.grad(uh)) + c * uh - f)**2 * dx

b_perp = ufl.conditional(ufl.eq(ufl.inner(b, b),0), b, -ufl.perp(b)/ufl.sqrt(ufl.inner(b,b)))
cross = abs(ufl.dot(b_perp, ufl.grad(uh)))
crosswind_loss = ufl.conditional(ufl.lt(cross, 1), 1/2*(5*cross**2 - 3*cross**3), ufl.sqrt(cross)) * dx
#loss = (uh-uD)**2 * ufl.dx
loss = residual + crosswind_loss
fs = FEniCSx_solver(pde_data=pde_data, loss_form=loss)

norm_b = ufl.sqrt(ufl.dot(b,b))
h = ufl.CellDiameter(domain=mesh) 
alpha = norm_b*h/(2*eps)
Xi = (1/ufl.tanh(alpha)-1/alpha)
tau_K = h/(2*norm_b)*Xi
Th = fem.functionspace(mesh, ('DG', 0))
tau = fem.Function(Th)
tau_exp = fem.Expression(tau_K, Th.element.interpolation_points())
tau.interpolate(tau_exp)
fs.set_weights(tau.x.array)

arr = np.array([])
upper = 100*fs.yh.x.array

/Users/vincentdallmer/dev/MA_ML_assisted_SUPG/utils/FEniCSx_solver.py:29: SparseEfficiencyWarning: splu converted its input to CSC format
  A_inv = scipy.sparse.linalg.splu(self._A_scipy)


In [2]:
from scipy.optimize import minimize
from dolfinx import io


def callback(intermediate_result):
    fval = intermediate_result.fun
    print(f"J: {fval}")


def eval(weights):
    fs.set_weights(weights=weights)
    return fs.loss()

def eval_grad(weights):
    return fs.grad()

bounds = scipy.optimize.Bounds(lb=np.zeros_like(fs.yh.x.array), ub=upper)
minimize(
    fun=eval,
    x0=fs.yh.x.array,
    jac=eval_grad,
    method='L-BFGS-B',
    #method='BFGS',
    tol=1e-9,
    options={"disp": False},
    callback=callback,
    bounds=bounds
)

J: 9.84387040994135
J: 9.190220753372909
J: 9.07673268305832
J: 8.946683639473775
J: 8.864390926533563
J: 8.828069603273207
J: 8.814443753575823
J: 8.792079237682653
J: 8.77043553339494
J: 8.75580427102776
J: 8.702261200595835
J: 8.659586288836918
J: 8.63475575817108
J: 8.580169318683
J: 8.540570775797637
J: 8.524986296498227
J: 8.523153438524663
J: 8.51358581549238
J: 8.510377194390642
J: 8.507531897002115
J: 8.504607652831881
J: 8.500787631975118
J: 8.494401389719336
J: 8.472885179885921
J: 8.463439894701029
J: 8.457860437312524
J: 8.452959962424206
J: 8.452161557573923
J: 8.45179787704717
J: 8.449672900378175
J: 8.446498293548856
J: 8.441930268801693
J: 8.437230677828616
J: 8.434828191810325
J: 8.43428815003351
J: 8.434236815817837
J: 8.43406878924532
J: 8.433102398168074
J: 8.432505014698275
J: 8.432008823098617
J: 8.428667089089283
J: 8.426123152061708
J: 8.424319737231023
J: 8.420403687282093
J: 8.41668073730164
J: 8.415702452793877
J: 8.415645642864977
J: 8.415621270947543
J: 8.

  message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
  success: True
   status: 0
      fun: 8.398634699499137
        x: [ 0.000e+00  0.000e+00 ...  0.000e+00  1.361e-04]
      nit: 612
      jac: [ 2.324e-01  6.214e-02 ...  2.137e-02 -6.831e-05]
     nfev: 649
     njev: 649
 hess_inv: <64x64 LbfgsInvHessProduct with dtype=float64>

In [5]:
ex_loss = fem.assemble_scalar(fem.form(ufl.replace(fs.loss_form, {fs.uh: uD})))
loss = fs.loss()
print(f"""
    loss exact solution: \t{ex_loss}\n
    loss approx. solution: \t{loss}
""")


    loss exact solution: 	11.111822389773018

    loss approx. solution: 	8.398634699499137



In [3]:

from utils.plotter import fem_plotter_grid
import pyvista as pv
pv.global_theme.cmap = 'coolwarm'

def interpolate_expr(expr, Wh):
    f = fem.Function(Wh)
    fem_expr = fem.Expression(expr, Wh.element.interpolation_points())
    f.interpolate(fem_expr)
    return f

test = ufl.TestFunction(fs.yh.function_space)
n = ufl.FacetNormal(mesh)
residual_test = (ufl.dot(b, ufl.grad(fs.uh))-f)**2 * test * dx 

b_perp = ufl.conditional(ufl.eq(ufl.inner(b, b),0), b, -ufl.perp(b)/ufl.sqrt(ufl.inner(b,b)))
cross = abs(ufl.dot(b_perp, ufl.grad(fs.uh)))
crosswind_loss = ufl.conditional(ufl.lt(cross, 1), 1/2*(5*cross**2 - 3*cross**3), ufl.sqrt(cross)) * test * dx

local_loss = fem.Function(fs.yh.function_space)

local_loss.x.array[:] = fem.assemble_vector(fem.form(residual_test)).array
#params = fem.Function(Wh)
#params.interpolate(fs.yh)
params= fs.yh
grid = fem_plotter_grid(Wh)
import pyvista as pv

p1, p2 = pv.Plotter(), pv.Plotter()

grid.add_data(fs.uh)
p1.add_mesh(
    grid.grid.warp_by_scalar(), 
    show_edges=True
)


grid.add_data(local_loss)
p2.add_mesh(
    grid.grid,
    show_edges=True
)
p2.camera_position = 'xy'
p1.show()
p2.show()

Widget(value='<iframe src="http://localhost:54020/index.html?ui=P_0x328dcf8c0_0&reconnect=auto" class="pyvista…

Widget(value='<iframe src="http://localhost:54020/index.html?ui=P_0x328e9ad50_1&reconnect=auto" class="pyvista…

In [14]:
from dolfinx import io
with io.XDMFFile(MPI.COMM_WORLD, "data/XMDF_files/cylinder_q1_8x8_constrained.xdmf", "w") as xdmf:
    mesh.name = 'q1_8x8'
    xdmf.write_mesh(mesh)
    fun = interpolate_expr(f, Wh)
    fun.name = 'f'
    uh = fs.uh
    uh.name = 'uh'
    uD.name = 'uD'
    params.name = 'params'
    local_loss.name = 'loss'
    xdmf.write_function(u=uh)
    xdmf.write_function(u=uD)
    xdmf.write_function(u=fun)